# APIs Lab
In this lab we will practice using APIs to retrieve and store data.

In [3]:
# Imports at the top
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt
%matplotlib inline

## Exercise 1: Get Data From Sheetsu

[Sheetsu](https://sheetsu.com/) is an online service that allows you to access any Google spreadsheet from an API. This can be a very powerful way to share a dataset with colleagues as well as to create a mini centralized data storage, that is simpler to edit than a database.

A Google Spreadsheet with Wine data can be found [here]().

It can be accessed through sheetsu API at this endpoint: https://sheetsu.com/apis/v1.0/dab55afd

Questions:

1. Use the requests library to access the document. Inspect the response text. What kind of data is it?

> Answer: it's a json string
- Check the status code of the response object. What code is it?
> 200
- use the appropriate libraries and read functions to read the response into a Pandas Dataframe
> Possible answers include: pd.read_json and json.loads + pd.Dataframe
- once you've imported the data into a dataframe, check the value of the 5th line: what's the price? (should be 6)

In [4]:
response = requests.get("https://sheetsu.com/apis/v1.0/dab55afd")

In [16]:
response.json()[1]

{u'Color': u'W',
 u'Consumed In': u'2015',
 u'Country': u'France',
 u'Grape': u'',
 u'Name': u'',
 u'Price': u'17.8',
 u'Region': u'France',
 u'Score': u'3',
 u'Vintage': u'2013',
 u'Vinyard': u'Peyruchet'}

### Exercise 2: Post Data to Sheetsu
Now that we've learned how to read data, it'd be great if we could also write data. For this we will need to use a _POST_ request.

1. Use the post command to add the following data to the spreadsheet:

In [58]:
post_data = {
'Grape' : "Lenny's specific grape wine 1607"
, 'Name' : 'My wonderful wine'
, 'Color' : 'R'
, 'Country' : 'US'
, 'Region' : 'Sonoma'
, 'Vinyard' : ''
, 'Score' : '10'
, 'Consumed In' : '2015'
, 'Vintage' : '1973'
, 'Price' : '200'
}

requests.post("https://sheetsu.com/apis/v1.0/dab55afd",data=post_data)

<Response [201]>

In [59]:
requests.get("https://sheetsu.com/apis/v1.0/dab55afd").json()[-1]

{u'Color': u'R',
 u'Consumed In': u'2015',
 u'Country': u'US',
 u'Grape': u"Lenny's specific grape wine 1607",
 u'Name': u'My wonderful wine',
 u'Price': u'200',
 u'Region': u'Sonoma',
 u'Score': u'10',
 u'Vintage': u'1973',
 u'Vinyard': u''}

1. What status did you get? How can you check that you actually added the data correctly?
- In this exercise, your classmates are adding data to the same spreadsheet. What happens because of this? Is it a problem? How could you mitigate it?

## Exercise 3: Data munging

Get back to the dataframe you've created in the beginning. Let's do some data munging:

1. Search For missing data
    - Is there any missing data? How do you deal with it?
    - Is there any data you can just remove?
    - Are the data types appropriate?
- Summarize the data 
    - Try using describe, min, max, mean, var

In [26]:
df = pd.read_json("https://sheetsu.com/apis/v1.0/dab55afd")

In [50]:
df_1 = df.applymap(lambda x: None if len(x.strip(' '))==0 else x)

In [55]:
df_1.isnull().sum()

Color           0
Consumed In     0
Country         2
Grape          99
Name            9
Price           6
Region          1
Score           1
Vintage         0
Vinyard        86
dtype: int64

In [57]:
for col in df_1.columns:
    print df_1[col].value_counts()

R             101
W              16
Z               2
Robin Hood      1
P               1
Name: Color, dtype: int64
2015                102
2014                  7
2013                  4
large quantities      2
Always                2
2012                  1
2098                  1
2016                  1
15                    1
Name: Consumed In, dtype: int64
US          101
France        9
Italy         3
Portugal      2
Spain         2
Oregon        1
666           1
Name: Country, dtype: int64
sauvignon blanc                                                                    4
grape drank                                                                        2
merlot, cab                                                                        2
sangiovese                                                                         2
pinot noir                                                                         1
merlot                                                                 

## Exercise 4: Feature Extraction

We would like to use a regression tree to predict the score of a wine. In order to do that, we first need to select and engineer appropriate features.

1. set the target to be the Score column, drop the rows with no score
- use pd.get_dummies to create dummy features for all the text columns
- fill the nan values in the numerical columns, using an appropriate method
- train a Decision tree regressor on the Score, using a train test split:
        X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.3, random_state=42)
- plot the test values, the predicted values and the residuals
- calculate R^2 score
- discuss your findings


## Exercise 5: IMDB Movies

Sometimes an API doesn't provide all the information we would like to get and we need to be creative.
Here we will use a combination of scraping and API calls to investigate the ratings and gross earnings of famous movies.

## 5.a Get top movies

The Internet Movie Database contains data about movies. Unfortunately it does not have a public API.

The page http://www.imdb.com/chart/top contains the list of the top 250 movies of all times. Retrieve the page using the requests library and then parse the html to obtain a list of the `movie_ids` for these movies. You can parse it with regular expression or using a library like `BeautifulSoup`.

**Hint:** movie_ids look like this: `tt2582802`

## 5.b Get top movies data

Although the Internet Movie Database does not have a public API, an open API exists at http://www.omdbapi.com.

Use this API to retrieve information about each of the 250 movies you have extracted in the previous step.
1. Check the documentation of omdbapi.com to learn how to request movie data by id
- define a function that returns a python object with all the information for a given id
- iterate on all the IDs and store the results in a list of such objects
- create a Pandas Dataframe from the list

## 5.c Get gross data

The OMDB API is great, but it does not provide information about Gross Revenue of the movie. We'll revert back to scraping for this.

1. Write a function that retrieves the gross revenue from the entry page at imdb.com
- the function should handle the exception of when the page doesn't report gross revenue
- retrieve the gross revenue for each movie and store it in a separate dataframe

## 5.d Data munging

1. Now that you have movie information and gross revenue information, let's clean the two datasets.
- check if there are null values. Be careful they may appear to be valid strings.
- convert the columns to the appropriate formats. In particular handle:
    - Released
    - Runtime
    - year
    - imdbRating
    - imdbVotes
- merge the data from the two datasets into a single one

## 5.d Text vectorization

There are several columns in the data that contain a comma separated list of items, for example the Genre column and the Actors column. Let's transform those to binary columns using the count vectorizer from scikit learn.

Append these columns to the merged dataframe.

**Hint:** In order to get the actors name right, you'll have to modify the `token_pattern` in the `CounteVectorizer`.

## Bonus: Final Questions:

1. what are the top 10 grossing movies?
- who are the 10 actors that appear in the most movies?
- what's the average grossing of the movies in which each of these actors appear?
- what genre is the oldest movie?
